In [41]:
import requests
from datetime import datetime, timedelta
import pandas as pd
import json

# constants
# care token might expire after several minutes (will do an error 401)
TOKEN = "BQCVzpWYeRrTmaGQ2fgzQ0NVnZoD_4B7VxqkWfzFBgVA-Ym3KHq-0b1fLLzMP-_nz-MkBRBx-2hCftpBiM5c5Of8cSUrds1p_qNxndNSUMaC9kM7IR29ubdfuuf1xM9X3x2hbDMAsfnIDOHu_Su7VpuWU43rXaerNqF-8InUSKS23-vc4hk"

if __name__ == "__main__":

    # time
    yesterday_datetime = datetime.now() - timedelta(days=1)
    yesterday_timestamp_milliseconds = int(yesterday_datetime.timestamp()) * 1000
    yesterday_timestamp_milliseconds = int(yesterday_datetime.timestamp()) * 1000

    # headers of our request
    headers = {
        "Accept" : "application/json",
        "Content-Type" : "application/json",
        "Authorization" : "Bearer {token}".format(token=TOKEN)
    }

    # making our request
    r = requests.get("https://api.spotify.com/v1/me/player/recently-played?before={time}".format(time=yesterday_timestamp_milliseconds), headers = headers)
    data = r.json()
    #r.status_code

    # creating lists, we'll fill them with the data from the json
    artist_names = []
    song_names = []
    played_at = []
    timestamps = []

    # loop to append our data
    for songs in data["items"]:
        artist_names.append(songs["track"]["album"]["artists"][0]["name"])
        song_names.append(songs["track"]["name"])
        played_at.append(songs["played_at"])
        timestamps.append(songs["played_at"][0:10])

    # storing the data into a dictionnary
    dictionary = {
    "artist_names":artist_names,
    "song_names":song_names,
    "played_at":played_at,
    "timestamps":timestamps
    }

    # converting my dictionnary into a dataframe
    df = pd.DataFrame(dictionary)

df

,artist_names,song_names,played_at,timestamps
0,James Newton Howard,London,2022-11-30T17:10:34.519Z,2022-11-30
1,Booba,Comme une étoile,2022-11-30T17:04:43.839Z,2022-11-30
2,Gold,Capitaine abandonné,2022-11-27T09:33:24.093Z,2022-11-27
3,Gold,Laissez Nous Chanter - 2017 Remastered,2022-11-27T09:29:31.534Z,2022-11-27
4,Jeanne Mas,En rouge et noir,2022-11-25T14:54:29.843Z,2022-11-25
5,Daniel Balavoine,L'Aziza,2022-11-25T14:50:31.790Z,2022-11-25
6,Vangelis,Anthem (The 2002 FIFA World Cup Official Anthe...,2022-11-25T14:45:20.252Z,2022-11-25
7,Julie Pietri,Eve lève-toi,2022-11-25T14:34:14.852Z,2022-11-25
8,Jean-Michel Jarre,Fourth Rendez-Vous,2022-11-25T14:28:53.390Z,2022-11-25
9,Mylène Farmer,C'est une belle journée,2022-11-25T11:52:55.493Z,2022-11-25
